In [ ]:
from flask import Flask, request, jsonify
import tableauserverclient as TSC
import logging

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

app = Flask(__name__)

# Tableau Server configuration
server_url = 'https://prod-useast-b.online.tableau.com'
token_name = "PowerAutomateAgent"
personal_access_token = "IK9CGGROTHCkI0DY2sslXQ==:FztEDBxDDAQY5gm8VL2J0LUTBRDfkQGq"  # Replace with your actual Token Secret
site = 'axosfinancialproduction'

# Initialize Tableau Server connection
try:
    tableau_auth = TSC.PersonalAccessTokenAuth(token_name, personal_access_token, site)
    server = TSC.Server(server_url, use_server_version=True)
    logger.info("Tableau Server connection initialized successfully.")
except Exception as e:
    logger.error(f"Failed to initialize Tableau Server connection: {str(e)}")
    raise

# Search function
def search_workbooks(query):
    try:
        with server.auth.sign_in(tableau_auth):
            logger.info("Successfully signed in to Tableau Server.")
            all_workbooks = list(TSC.Pager(server.workbooks))
            logger.info(f"Found {len(all_workbooks)} total workbooks.")
            results = []
            for workbook in all_workbooks:
                server.workbooks.populate_views(workbook)
                if (query.lower() in workbook.name.lower() or
                    any(query.lower() in view.name.lower() for view in workbook.views)):
                    results.append({
                        "name": workbook.name,
                        "id": workbook.id,
                        "project_name": workbook.project_name,
                        "webpage_url": workbook.webpage_url,
                        "views": [view.name for view in workbook.views]
                    })
            return results
    except Exception as e:
        logger.error(f"Error in search_workbooks: {str(e)}")
        raise

@app.route('/search', methods=['GET'])
def search():
    try:
        query = request.args.get('query', default="", type=str)
        if not query:
            logger.warning("No query parameter provided.")
            return jsonify({"error": "No query parameter provided"}), 400
        logger.info(f"Received search query: {query}")
        results = search_workbooks(query)
        logger.info(f"Returning {len(results)} results for query: {query}")
        return jsonify(results)
    except Exception as e:
        logger.error(f"Error in /search endpoint: {str(e)}")
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    try:
        app.run(debug=False, host='0.0.0.0', port=5000)  # Changed debug to False
    except Exception as e:
        logger.error(f"Failed to start Flask server: {str(e)}")
        raise

DEBUG:TSC:request method get, url: https://prod-useast-b.online.tableau.com/api/2.4/serverInfo
DEBUG:TSC:[12:51:33] Begin blocking request to https://prod-useast-b.online.tableau.com/api/2.4/serverInfo
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): prod-useast-b.online.tableau.com:443
DEBUG:urllib3.connectionpool:https://prod-useast-b.online.tableau.com:443 "GET /api/2.4/serverInfo HTTP/11" 200 None
DEBUG:TSC:[12:51:34] Call finished
DEBUG:TSC:[12:51:34] Async request returned: received <Response [200]>
DEBUG:TSC:Response status: <Response [200]>
DEBUG:TSC:Server response from https://prod-useast-b.online.tableau.com/api/2.4/serverInfo
INFO:TSC:versions: 3.25, 2.4
INFO:__main__:Tableau Server connection initialized successfully.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.20.4.160:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.20.4.160:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:Received search query: Consumer
DEBUG:urllib3.connectionpool:https://prod-useast-b.online.tableau.com:443 "POST /api/3.25/auth/signin HTTP/11" 200 None
DEBUG:TSC:Response status: <Response [200]>
INFO:TSC:Signed into https://prod-useast-b.online.tableau.com as user with id 10ce16df-26a9-47c2-9513-71cfcc992a12
INFO:__main__:Successfully signed in to Tableau Server.
INFO:TSC:Querying all workbooks on site
DEBUG:TSC:request method get, url: https://prod-useast-b.online.tableau.com/api/3.25/sites/438dd919-a45c-4cb8-928a-12c68660c061/workbooks
DEBUG:TSC:[12:55:58] Begin b